In [39]:
import pandas as pd
import altair as alt
from google.cloud import bigquery
import geopandas as gpd
from shapely import wkt
#import google.auth
#from google.cloud import bigquery
#from google.cloud import bigquery_storage
bqclient = bigquery.Client.from_service_account_json('C:/Users/bennd/Documents/MUSA509/TransitPolicyApp-99838a65a6ed.json')
pd.set_option('mode.chained_assignment', None)
#ridership = pd.read_csv("data/ridership.csv")
#ridership['month'] = pd.to_datetime(ridership['month'])
covid = pd.read_csv("data/us-counties.csv")
covid['date'] = pd.to_datetime(covid['date'])
top50 = pd.read_csv("data/top50.csv")

In [52]:
covid_l = covid[covid["date"] == "2020-09-30"]
covid_l.head()

,date,county,state,fips,cases,deaths
577661,2020-09-30,Autauga,Alabama,1001,1791,27.0
577662,2020-09-30,Baldwin,Alabama,1003,5640,52.0
577663,2020-09-30,Barbour,Alabama,1005,896,7.0
577664,2020-09-30,Bibb,Alabama,1007,664,11.0
577665,2020-09-30,Blount,Alabama,1009,1629,15.0


In [53]:
top50_c = top50.merge(covid_l, on="fips", how='left')
top50_c.head()

,GEOID10,order,metro_area,fips,date,county,state,cases,deaths
0,10162,1,"Bridgeport-Stamford, CT-NY",9001,2020-09-30,Fairfield,Connecticut,20225,1422.0
1,11350,2,"Buffalo, NY",36029,2020-09-30,Erie,New York,11500,625.0
2,15670,3,"Charlotte, NC-SC",37119,2020-09-30,Mecklenburg,North Carolina,28975,360.0
3,16264,4,"Chicago, IL-IN",17031,2020-09-30,Cook,Illinois,145462,5225.0
4,16885,5,"Cincinnati, OH-KY-IN",39061,2020-09-30,Hamilton,Ohio,13426,320.0


In [54]:
ridership_agg = ridership[ridership["month"] == "2020-09-01"]
ridership_agg = ridership_agg.groupby(['GEOID10'], as_index=False).sum()
ridership_agg.head()

,GEOID10,OID,trips,ctr_lat,ctr_lon
0,10162,24937,561267,82.470308,-146.618753
1,10972,2247,41808,51.933844,-194.948860
2,11350,32037,1031785,128.779884,-236.471558
3,11755,31596,110494,133.442927,-219.485590
4,13375,7091,108702,81.657192,-162.860484


In [55]:
ridership_agg = ridership_agg[['GEOID10','trips']]
top50_c = top50_c.merge(ridership_agg, on="GEOID10", how='left')
top50_c.head()

,GEOID10,order,metro_area,fips,date,county,state,cases,deaths,trips
0,10162,1,"Bridgeport-Stamford, CT-NY",9001,2020-09-30,Fairfield,Connecticut,20225,1422.0,561267
1,11350,2,"Buffalo, NY",36029,2020-09-30,Erie,New York,11500,625.0,1031785
2,15670,3,"Charlotte, NC-SC",37119,2020-09-30,Mecklenburg,North Carolina,28975,360.0,750275
3,16264,4,"Chicago, IL-IN",17031,2020-09-30,Cook,Illinois,145462,5225.0,15097626
4,16885,5,"Cincinnati, OH-KY-IN",39061,2020-09-30,Hamilton,Ohio,13426,320.0,640183


In [56]:
alt.Chart(top50_c).mark_circle(size=200).encode(
    x=alt.X('cases', axis=alt.Axis(title='COVID-19 Cases (Cumulative)')),
    y=alt.Y('trips', axis=alt.Axis(title='Total Transit Trips in September 2020')),
    color=alt.Color('state', legend=None),
    tooltip=['metro_area', 'cases', 'deaths', 'trips']
).interactive()

alt.Chart(...)

In [64]:
search_county = "San Francisco"
covid_s = covid[covid["county"] == search_county]
covid_s.head()

,date,county,state,fips,cases,deaths
49,2020-02-02,San Francisco,California,6075,2,0.0
57,2020-02-03,San Francisco,California,6075,2,0.0
65,2020-02-04,San Francisco,California,6075,2,0.0
73,2020-02-05,San Francisco,California,6075,2,0.0
82,2020-02-06,San Francisco,California,6075,2,0.0


In [65]:
covid_s['new_cases'] = covid_s['cases'].diff()
covid_s['new_deaths'] = covid_s['deaths'].diff()
covid_s['cases_avg'] = covid_s.iloc[:,6].rolling(window=7).mean()
covid_s['deaths_avg'] = covid_s.iloc[:,7].rolling(window=7).mean()
covid_s

,date,county,state,fips,cases,deaths,new_cases,new_deaths,cases_avg,deaths_avg
49,2020-02-02,San Francisco,California,6075,2,0.0,NaN,NaN,NaN,NaN
57,2020-02-03,San Francisco,California,6075,2,0.0,0.0,0.0,NaN,NaN
65,2020-02-04,San Francisco,California,6075,2,0.0,0.0,0.0,NaN,NaN
73,2020-02-05,San Francisco,California,6075,2,0.0,0.0,0.0,NaN,NaN
82,2020-02-06,San Francisco,California,6075,2,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
761167,2020-11-26,San Francisco,California,6075,15056,160.0,179.0,0.0,142.285714,0.571429
764385,2020-11-27,San Francisco,California,6075,15159,160.0,103.0,0.0,129.285714,0.571429
767603,2020-11-28,San Francisco,California,6075,15380,160.0,221.0,0.0,150.000000,0.285714
770821,2020-11-29,San Francisco,California,6075,15453,160.0,73.0,0.0,143.571429,0.285714


In [70]:
alt.Chart(covid_s).mark_bar().encode(
    x=alt.X('date:T', axis=alt.Axis(title='Month')),
    y=alt.Y('cases_avg:Q', axis=alt.Axis(title='New COVID-19 Cases Reported')),
)

alt.Chart(...)

In [77]:
alt.Chart(covid_s).mark_bar().encode(
    x=alt.X('date:T', axis=alt.Axis(title='Month')),
    y=alt.Y('deaths_avg:Q', axis=alt.Axis(title='New COVID-19 Deaths Reported')),
    color=alt.value('orange'),
)

alt.Chart(...)

In [40]:
covid_s.dtypes

date           object
county         object
state          object
fips          float64
cases           int64
deaths        float64
new_cases     float64
new_deaths    float64
cases_avg     float64
deaths_avg    float64
dtype: object

In [41]:
covid_s['DateTime'] = pd.to_datetime(covid_s['date'])

In [42]:
covid_s.head()

,date,county,state,fips,cases,deaths,new_cases,new_deaths,cases_avg,deaths_avg,DateTime
5151,2020-03-19,Ozaukee,Wisconsin,55089.0,3,1.0,NaN,NaN,NaN,NaN,2020-03-19
6063,2020-03-20,Ozaukee,Wisconsin,55089.0,7,1.0,4.0,0.0,NaN,NaN,2020-03-20
7096,2020-03-21,Ozaukee,Wisconsin,55089.0,11,1.0,4.0,0.0,NaN,NaN,2020-03-21
8234,2020-03-22,Ozaukee,Wisconsin,55089.0,13,1.0,2.0,0.0,NaN,NaN,2020-03-22
9460,2020-03-23,Ozaukee,Wisconsin,55089.0,14,1.0,1.0,0.0,NaN,NaN,2020-03-23


In [43]:
covid_s.dtypes

date                  object
county                object
state                 object
fips                 float64
cases                  int64
deaths               float64
new_cases            float64
new_deaths           float64
cases_avg            float64
deaths_avg           float64
DateTime      datetime64[ns]
dtype: object

In [54]:
base = alt.Chart(covid_s).encode(
    alt.X('date:T', axis=alt.Axis(title=None))
)

line_c = base.mark_bar(stroke='#5276A7', interpolate='monotone').encode(
    alt.Y('cases_avg',
          axis=alt.Axis(title='Average Cases', titleColor='#5276A7'))
)

line_d = base.mark_bar(stroke='#5276A7', interpolate='monotone').encode(
    alt.Y('deaths_avg',
          axis=alt.Axis(title='Deatgs Cases', titleColor='#5276A7'))
)

alt.layer(line_c, line_d).resolve_scale(
    y = 'independent'
)

alt.LayerChart(...)

In [4]:
covid.head()

,date,county,state,fips,cases,deaths
0,1/21/2020,Snohomish,Washington,53061,1,0.0
1,1/22/2020,Snohomish,Washington,53061,1,0.0
2,1/23/2020,Snohomish,Washington,53061,1,0.0
3,1/24/2020,Cook,Illinois,17031,1,0.0
4,1/24/2020,Snohomish,Washington,53061,1,0.0


In [5]:
covid_l = covid[covid["date"] == "12/01/2020"]
covid_l.head()

,date,county,state,fips,cases,deaths


In [66]:
top50 = pd.read_csv("data/top50.csv")
top50.head()

,GEOID10,order,metro_area,fips
0,10162,1,"Bridgeport-Stamford, CT-NY",9001
1,11350,2,"Buffalo, NY",36029
2,15670,3,"Charlotte, NC-SC",37119
3,16264,4,"Chicago, IL-IN",17031
4,16885,5,"Cincinnati, OH-KY-IN",39061


In [40]:
#Pull ridership data from Google BigQuery

query = f"""
    SELECT metro_area, GEOID10, mode, month, trips, classification, ctr_lat, ctr_lon
    FROM `transitpolicyapp.ridership.Ridership`
    GROUP BY metro_area, GEOID10, mode, month, trips, classification, ctr_lat, ctr_lon
"""
ridership = bqclient.query(query).to_dataframe()
ridership.head()

,metro_area,GEOID10,mode,month,trips,classification,ctr_lat,ctr_lon
0,"Monessen-California, PA",58168,Bus,2019-01-01 00:00:00+00:00,22447,Non-Rail,40.136158,-79.884233
1,"Monessen-California, PA",58168,Bus,2019-02-01 00:00:00+00:00,23338,Non-Rail,40.136158,-79.884233
2,"Monessen-California, PA",58168,Bus,2019-03-01 00:00:00+00:00,24577,Non-Rail,40.136158,-79.884233
3,"Monessen-California, PA",58168,Bus,2019-04-01 00:00:00+00:00,24723,Non-Rail,40.136158,-79.884233
4,"Monessen-California, PA",58168,Bus,2019-05-01 00:00:00+00:00,23115,Non-Rail,40.136158,-79.884233


In [47]:
#Generate interactive altair ridership chart by mode and month

search = "Philadelphia, PA-NJ-DE-MD"

ridership_s = ridership[ridership["metro_area"] == search]
alt.Chart(ridership_s).mark_area().encode(
    x=alt.X('month:T', axis=alt.Axis(title='Month')),
    y=alt.Y('trips:Q', axis=alt.Axis(title='Total Transit Trips')),
    color="mode:N",
    tooltip=['month','mode', 'trips']
).interactive()

alt.Chart(...)